In [22]:
import pandas as pd
import numpy as np

from pysal.lib import weights
from libpysal.weights import Kernel
import networkx as nx
import geopandas
from node2vec import Node2Vec
from shapely.geometry import Point

In [2]:
locations = geopandas.read_file('zip:///home/abbas/hybrid_model/data/taxi_zones.zip')
print(locations)

     OBJECTID  Shape_Leng  Shape_Area                     zone  LocationID  \
0           1    0.116357    0.000782           Newark Airport           1   
1           2    0.433470    0.004866              Jamaica Bay           2   
2           3    0.084341    0.000314  Allerton/Pelham Gardens           3   
3           4    0.043567    0.000112            Alphabet City           4   
4           5    0.092146    0.000498            Arden Heights           5   
..        ...         ...         ...                      ...         ...   
258       259    0.126750    0.000395       Woodlawn/Wakefield         259   
259       260    0.133514    0.000422                 Woodside         260   
260       261    0.027120    0.000034       World Trade Center         261   
261       262    0.049064    0.000122           Yorkville East         262   
262       263    0.037017    0.000066           Yorkville West         263   

           borough                                           ge

In [25]:
def transformGeometryToVector(dataset, attributeId):
    
    # w = libpysal.weights.DistanceBand.from_dataframe(self.data, threshold=50000, binary=False)
    # print(w.islands)

    w = Kernel.from_dataframe(dataset, fixed=False, function="gaussian")
    # w = KNN.from_dataframe(self.data, k=5)
    nodes = w.weights.keys()
    edges = [(node, neighbour) for node in nodes for neighbour in w[node]]
    my_graph = nx.Graph(edges)

    dimensions = 64
    node2vec = Node2Vec(
        my_graph, dimensions=dimensions, walk_length=15, num_walks=100, workers=1
    )
    model = node2vec.fit(window=10, min_count=1, batch_words=4)

    dataset[attributeId] = ""
    for index, _ in dataset.iterrows():
        dataset.at[index, attributeId] = model.wv[str(index)]

    return dataset



In [ ]:
new_locations = transformGeometryToVector(locations, 'geometry_embed')
print(new_locations)

In [14]:
citi_data = pd.read_csv("/home/abbas/hybrid_model/data/202011-citibike-tripdata.csv")
#print(citi_data.head())
#print(citi_data.shape)
citi_data_sample = tlc_data.sample(n=1000, random_state=1)
print(citi_data_sample)


         tripduration                 starttime                  stoptime  \
376491           1330  2020-11-07 09:19:50.3460  2020-11-07 09:42:00.8980   
824840           1658  2020-11-13 16:43:19.9780  2020-11-13 17:10:58.4650   
1600485           330  2020-11-27 19:19:01.1630  2020-11-27 19:24:31.2440   
23617             404  2020-11-01 15:52:04.7940  2020-11-01 15:58:49.1250   
858316           2154  2020-11-14 10:13:33.3580  2020-11-14 10:49:28.0730   
...               ...                       ...                       ...   
332474            558  2020-11-06 16:16:32.5880  2020-11-06 16:25:51.3310   
494270           2540  2020-11-08 12:52:29.3780  2020-11-08 13:34:49.4270   
1735515           377  2020-11-30 22:11:57.6180  2020-11-30 22:18:14.6320   
1661793          4648  2020-11-29 01:49:03.0920  2020-11-29 03:06:31.5010   
1169712           485  2020-11-19 18:12:13.7970  2020-11-19 18:20:19.2550   

         start station id                start station name  \
376491      

In [23]:
def coordinateToPoint(attributeId):
    citi_data_sample[attributeId] = ""
    for index, value in citi_data_sample.iterrows():
        #print(value['end station longitude'])
        citi_data_sample.at[index, attributeId] = Point(value['end station latitude'], value['end station longitude'])
        
coordinateToPoint('geometry')
print(citi_data_sample)

         tripduration                 starttime                  stoptime  \
376491           1330  2020-11-07 09:19:50.3460  2020-11-07 09:42:00.8980   
824840           1658  2020-11-13 16:43:19.9780  2020-11-13 17:10:58.4650   
1600485           330  2020-11-27 19:19:01.1630  2020-11-27 19:24:31.2440   
23617             404  2020-11-01 15:52:04.7940  2020-11-01 15:58:49.1250   
858316           2154  2020-11-14 10:13:33.3580  2020-11-14 10:49:28.0730   
...               ...                       ...                       ...   
332474            558  2020-11-06 16:16:32.5880  2020-11-06 16:25:51.3310   
494270           2540  2020-11-08 12:52:29.3780  2020-11-08 13:34:49.4270   
1735515           377  2020-11-30 22:11:57.6180  2020-11-30 22:18:14.6320   
1661793          4648  2020-11-29 01:49:03.0920  2020-11-29 03:06:31.5010   
1169712           485  2020-11-19 18:12:13.7970  2020-11-19 18:20:19.2550   

         start station id                start station name  \
376491      

In [26]:
new_citi_data = transformGeometryToVector(citi_data_sample, 'geometry_embed')
print(new_citi_data)

/storage/home/abbas/notebook/jupyterenv/lib/python3.7/site-packages/libpysal/weights/util.py:1073: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  data = np.vstack([np.array(shape.centroid) for shape in first_choice])
/storage/home/abbas/notebook/jupyterenv/lib/python3.7/site-packages/libpysal/weights/distance.py:662: RuntimeWarning: invalid value encountered in true_divide
  zi = np.array([dict(list(zip(ni, di)))[nid] for nid in nids]) / bw[i]
/storage/home/abbas/notebook/jupyterenv/lib/python3.7/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 159 disconnected components.
  warnings.warn(message)


Computing transition probabilities:   0%|          | 0/1000 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:57<00:00,  1.74it/s]


         tripduration                 starttime                  stoptime  \
376491           1330  2020-11-07 09:19:50.3460  2020-11-07 09:42:00.8980   
824840           1658  2020-11-13 16:43:19.9780  2020-11-13 17:10:58.4650   
1600485           330  2020-11-27 19:19:01.1630  2020-11-27 19:24:31.2440   
23617             404  2020-11-01 15:52:04.7940  2020-11-01 15:58:49.1250   
858316           2154  2020-11-14 10:13:33.3580  2020-11-14 10:49:28.0730   
...               ...                       ...                       ...   
332474            558  2020-11-06 16:16:32.5880  2020-11-06 16:25:51.3310   
494270           2540  2020-11-08 12:52:29.3780  2020-11-08 13:34:49.4270   
1735515           377  2020-11-30 22:11:57.6180  2020-11-30 22:18:14.6320   
1661793          4648  2020-11-29 01:49:03.0920  2020-11-29 03:06:31.5010   
1169712           485  2020-11-19 18:12:13.7970  2020-11-19 18:20:19.2550   

         start station id                start station name  \
376491      